In [2]:
from collections import OrderedDict
from fastprogress import progress_bar
from pathlib import Path
from sklearn.model_selection import train_test_split, ShuffleSplit
from torch import nn
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter

import functools
import os
import pandas as pd
import random
import shutil
import torch
import torch.nn.functional as F


target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']
input_cols = ['sequence', 'structure', 'predicted_loop_type']
error_cols = ['reactivity_error', 'deg_error_Mg_pH10', 'deg_error_Mg_50C', 'deg_error_pH10', 'deg_error_50C']

token_dicts = {
    "sequence": {x: i for i, x in enumerate("ACGU")},
    "structure": {x: i for i, x in enumerate('().')},
    "predicted_loop_type": {x: i for i, x in enumerate("BEHIMSX")}
}


def set_seed(seed: int = 1042777):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

### Loader


In [3]:
from sklearn.model_selection import train_test_split, ShuffleSplit
from torch import nn
from torch.utils.data import Dataset

import functools


BASE_PATH = "/kaggle/input/stanford-covid-vaccine"
MODEL_SAVE_PATH = "/kaggle/model"


def preprocess_inputs(df, cols):
    return np.concatenate([preprocess_feature_col(df, col) for col in cols], axis=2)


def preprocess_feature_col(df, col):
    dic = token_dicts[col]
    dic_len = len(dic)
    seq_length = len(df[col][0])
    ident = np.identity(dic_len)
    # convert to one hot
    arr = np.array(
        df[[col]].applymap(lambda seq: [ident[dic[x]] for x in seq]).values.tolist()
    ).squeeze(1)
    # shape: data_size x seq_length x dic_length
    assert arr.shape == (len(df), seq_length, dic_len)
    return arr


def preprocess(base_data, is_test=False):
    inputs = preprocess_inputs(base_data, input_cols)
    if is_test:
        labels = None
    else:
        labels = np.array(base_data[target_cols].values.tolist()).transpose((0, 2, 1))
        assert labels.shape[2] == len(target_cols)
    assert inputs.shape[2] == 14
    return inputs, labels


def get_bpp_feature(bpp):
    bpp_nb_mean = 0.077522  # mean of bpps_nb across all training data
    bpp_nb_std = 0.08914  # std of bpps_nb across all training data
    bpp_max = bpp.max(-1)[0]
    bpp_sum = bpp.sum(-1)
    bpp_nb = torch.true_divide((bpp > 0).sum(dim=1), bpp.shape[1])
    bpp_nb = torch.true_divide(bpp_nb - bpp_nb_mean, bpp_nb_std)
    return [bpp_max.unsqueeze(2), bpp_sum.unsqueeze(2), bpp_nb.unsqueeze(2)]


@functools.lru_cache(5000)
def load_from_id(id_):
    path = Path(BASE_PATH) / f"bpps/{id_}.npy"
    data = np.load(str(path))
    return data


def get_distance_matrix(leng):
    idx = np.arange(leng)
    Ds = []
    for i in range(len(idx)):
        d = np.abs(idx[i] - idx)
        Ds.append(d)

    Ds = np.array(Ds) + 1
    Ds = 1 / Ds
    Ds = Ds[None, :, :]
    Ds = np.repeat(Ds, 1, axis=0)

    Dss = []
    for i in [1, 2, 4]:
        Dss.append(Ds ** i)
    Ds = np.stack(Dss, axis=3)
    print(Ds.shape)
    return Ds


def get_structure_adj(df):
    Ss = []
    for i in range(len(df)):
        seq_length = df["seq_length"].iloc[i]
        structure = df["structure"].iloc[i]
        sequence = df["sequence"].iloc[i]

        cue = []
        a_structures = OrderedDict([
            (("A", "U"), np.zeros([seq_length, seq_length])),
            (("C", "G"), np.zeros([seq_length, seq_length])),
            (("U", "G"), np.zeros([seq_length, seq_length])),
            (("U", "A"), np.zeros([seq_length, seq_length])),
            (("G", "C"), np.zeros([seq_length, seq_length])),
            (("G", "U"), np.zeros([seq_length, seq_length])),
        ])
        for j in range(seq_length):
            if structure[j] == "(":
                cue.append(j)
            elif structure[j] == ")":
                start = cue.pop()
                a_structures[(sequence[start], sequence[j])][start, j] = 1
                a_structures[(sequence[j], sequence[start])][j, start] = 1

        a_strc = np.stack([a for a in a_structures.values()], axis=2)
        a_strc = np.sum(a_strc, axis=2, keepdims=True)
        Ss.append(a_strc)

    Ss = np.array(Ss)
    return Ss


def create_loader(df, batch_size=1, is_test=False):
    features, labels = preprocess(df, is_test)
    features_tensor = torch.from_numpy(features)
    if labels is not None:
        labels_tensor = torch.from_numpy(labels)
        dataset = VacDataset(features_tensor, df, labels_tensor)
        loader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True, drop_last=False)
    else:
        dataset = VacDataset(features_tensor, df, None)
        loader = torch.utils.data.DataLoader(dataset, batch_size, shuffle=False, drop_last=False)
    return loader


class VacDataset(Dataset):
    def __init__(self, features, df, labels=None):
        self.features = features
        self.labels = labels
        self.test = labels is None
        self.ids = df["id"]
        self.score = None
        self.structure_adj = get_structure_adj(df)
        self.distance_matrix = get_distance_matrix(self.structure_adj.shape[1])
        if "score" in df.columns:
            self.score = df["score"]
        else:
            df["score"] = 1.0
            self.score = df["score"]
        self.signal_to_noise = None
        if not self.test:
            self.signal_to_noise = df["signal_to_noise"]
            assert self.features.shape[0] == self.labels.shape[0]
        else:
            assert self.ids is not None

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        bpp = torch.from_numpy(load_from_id(self.ids[index]).copy()).float()
        adj = self.structure_adj[index]
        distance = self.distance_matrix[0]
        bpp = np.concatenate([bpp[:, :, None], adj, distance], axis=2)
        if self.test:
            return dict(sequence=self.features[index].float(), bpp=bpp, ids=self.ids[index])
        else:
            return dict(sequence=self.features[index].float(), bpp=bpp,
                        label=self.labels[index], ids=self.ids[index],
                        signal_to_noise=self.signal_to_noise[index],
                        score=self.score[index])


### Model

In [4]:
from torch.nn import TransformerEncoder, TransformerEncoderLayer

import math


class Conv1dStack(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size=3, padding=1, dilation=1):
        super(Conv1dStack, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv1d(in_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm1d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )
        self.res = nn.Sequential(
            nn.Conv1d(out_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm1d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )

    def forward(self, x):
        x = self.conv(x)
        h = self.res(x)
        return x + h


class Conv2dStack(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size=3, padding=1, dilation=1):
        super(Conv2dStack, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )
        self.res = nn.Sequential(
            nn.Conv2d(out_dim, out_dim, kernel_size=kernel_size, padding=padding, dilation=dilation, bias=False),
            nn.BatchNorm2d(out_dim),
            nn.Dropout(0.1),
            nn.LeakyReLU(),
        )

    def forward(self, x):
        x = self.conv(x)
        h = self.res(x)
        return x + h


class SeqEncoder(nn.Module):
    def __init__(self, in_dim: int):
        super(SeqEncoder, self).__init__()
        self.conv0 = Conv1dStack(in_dim, 128, 3, padding=1)
        self.conv1 = Conv1dStack(128, 64, 6, padding=5, dilation=2)
        self.conv2 = Conv1dStack(64, 32, 15, padding=7, dilation=1)
        self.conv3 = Conv1dStack(32, 32, 30, padding=29, dilation=2)

    def forward(self, x):
        x1 = self.conv0(x)
        x2 = self.conv1(x1)
        x3 = self.conv2(x2)
        x4 = self.conv3(x3)
        x = torch.cat([x1, x2, x3, x4], dim=1)
        # x = x.permute(0, 2, 1).contiguous()
        # BATCH x 256 x seq_length
        return x


class BppAttn(nn.Module):
    def __init__(self, in_channel: int, out_channel: int):
        super(BppAttn, self).__init__()
        self.conv0 = Conv1dStack(in_channel, out_channel, 3, padding=1)
        self.bpp_conv = Conv2dStack(5, out_channel)

    def forward(self, x, bpp):
        x = self.conv0(x)
        bpp = self.bpp_conv(bpp)
        # BATCH x C x SEQ x SEQ
        # BATCH x C x SEQ
        x = torch.matmul(bpp, x.unsqueeze(-1))
        return x.squeeze(-1)


class PositionalEncoding(nn.Module):

    def __init__(self, d_model, dropout=0.15, max_len=143):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)


class TransformerWrapper(nn.Module):
    def __init__(self, dmodel=256, nhead=8, num_layers=2):
        super(TransformerWrapper, self).__init__()
        self.pos_encoder = PositionalEncoding(256)
        encoder_layer = TransformerEncoderLayer(d_model=dmodel, nhead=nhead)
        self.transformer_encoder = TransformerEncoder(encoder_layer, num_layers)
        self.pos_emb = PositionalEncoding(dmodel)

    def flatten_parameters(self):
        pass

    def forward(self, x):
        x = x.permute((1, 0, 2)).contiguous()
        x = self.pos_emb(x)
        x = self.transformer_encoder(x)
        x = x.permute((1, 0, 2)).contiguous()
        return x, None


class RnnLayers(nn.Module):
    def __init__(self, dmodel, dropout=0.3, transformer_layers: int = 2):
        super(RnnLayers, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.rnn0 = TransformerWrapper(dmodel, nhead=8, num_layers=transformer_layers)
        self.rnn1 = nn.LSTM(dmodel, dmodel // 2, batch_first=True, num_layers=1, bidirectional=True)
        self.rnn2 = nn.GRU(dmodel, dmodel // 2, batch_first=True, num_layers=1, bidirectional=True)

    def forward(self, x):
        self.rnn0.flatten_parameters()
        x, _ = self.rnn0(x)
        if self.rnn1 is not None:
            self.rnn1.flatten_parameters()
            x = self.dropout(x)
            x, _ = self.rnn1(x)
        if self.rnn2 is not None:
            self.rnn2.flatten_parameters()
            x = self.dropout(x)
            x, _ = self.rnn2(x)
        return x

    
class BaseAttnModel(nn.Module):
    def __init__(self, transformer_layers: int = 2):
        super(BaseAttnModel, self).__init__()
        self.linear0 = nn.Linear(14 + 3, 1)
        self.seq_encoder_x = SeqEncoder(18)
        self.attn = BppAttn(256, 128)
        self.seq_encoder_bpp = SeqEncoder(128)
        self.seq = RnnLayers(256 * 2, dropout=0.3,
                             transformer_layers=transformer_layers)

    def forward(self, x, bpp):
        bpp_features = get_bpp_feature(bpp[:, :, :, 0].float())
        x = torch.cat([x] + bpp_features, dim=-1)
        learned = self.linear0(x)
        x = torch.cat([x, learned], dim=-1)
        x = x.permute(0, 2, 1).contiguous().float()
        # BATCH x 18 x seq_len
        bpp = bpp.permute([0, 3, 1, 2]).contiguous().float()
        # BATCH x 5 x seq_len x seq_len
        x = self.seq_encoder_x(x)
        # BATCH x 256 x seq_len
        bpp = self.attn(x, bpp)
        bpp = self.seq_encoder_bpp(bpp)
        # BATCH x 256 x seq_len
        x = x.permute(0, 2, 1).contiguous()
        # BATCH x seq_len x 256
        bpp = bpp.permute(0, 2, 1).contiguous()
        # BATCH x seq_len x 256
        x = torch.cat([x, bpp], dim=2)
        # BATCH x seq_len x 512
        x = self.seq(x)
        return x


class AEModel(nn.Module):
    def __init__(self, transformer_layers: int = 2):
        super(AEModel, self).__init__()
        self.seq = BaseAttnModel(transformer_layers=transformer_layers)
        self.linear = nn.Sequential(
            nn.Linear(256 * 2, 14),
            nn.Sigmoid(),
        )

    def forward(self, x, bpp):
        x = self.seq(x, bpp)
        x = F.dropout(x, p=0.3)
        x = self.linear(x)
        return x


class FromAeModel(nn.Module):
    def __init__(self, seq, pred_len=68, dmodel: int = 256):
        super(FromAeModel, self).__init__()
        self.seq = seq
        self.pred_len = pred_len
        self.linear = nn.Sequential(
            nn.Linear(dmodel * 2, len(target_cols)),
        )

    def forward(self, x, bpp):
        x = self.seq(x, bpp)
        x = self.linear(x)
        x = x[:, :self.pred_len]
        return x


In [5]:
base_train_data = pd.read_json(str(Path(BASE_PATH) / 'train.json'), lines=True)
base_train_data.head()

device = torch.device('cuda')
BATCH_SIZE = 64
base_train_data = pd.read_json(str(Path(BASE_PATH) / 'train.json'), lines=True)
base_test_data = pd.read_json(str(Path(BASE_PATH) / 'test.json'), lines=True)
public_df = base_test_data.query("seq_length == 107").copy()
private_df = base_test_data.query("seq_length == 130").copy()
print(f"public_df: {public_df.shape}")
print(f"private_df: {private_df.shape}")
public_df = public_df.reset_index()
private_df = private_df.reset_index()

features, _ = preprocess(base_train_data, True)
features_tensor = torch.from_numpy(features)
dataset0 = VacDataset(features_tensor, base_train_data, None)
features, _ = preprocess(public_df, True)
features_tensor = torch.from_numpy(features)
dataset1 = VacDataset(features_tensor, public_df, None)
features, _ = preprocess(private_df, True)
features_tensor = torch.from_numpy(features)
dataset2 = VacDataset(features_tensor, private_df, None)

loader0 = torch.utils.data.DataLoader(dataset0, BATCH_SIZE, shuffle=False, drop_last=False)
loader1 = torch.utils.data.DataLoader(dataset1, BATCH_SIZE, shuffle=False, drop_last=False)
loader2 = torch.utils.data.DataLoader(dataset2, BATCH_SIZE, shuffle=False, drop_last=False)

public_df: (629, 7)
private_df: (3005, 7)
(1, 107, 107, 3)
(1, 107, 107, 3)
(1, 130, 130, 3)


### Pretrain

In [6]:
def learn_from_batch_ae(model, data, device):
    seq = data["sequence"].clone()
    seq[:, :, :14] = F.dropout2d(seq[:, :, :14], p=0.3)
    target = data["sequence"][:, :, :14]
    out = model(seq.to(device), data["bpp"].to(device))
    loss = F.binary_cross_entropy(out, target.to(device))
    return loss


def train_ae(model, train_data, optimizer, lr_scheduler, epochs=10, device="cpu",
             start_epoch: int = 0, start_it: int = 0, log_path: str = "./logs"):
    print(f"device: {device}")
    losses = []
    it = start_it
    model_save_path = Path(MODEL_SAVE_PATH)
    start_epoch = start_epoch
    end_epoch = start_epoch + epochs
    min_loss = 10.0
    min_loss_epoch = 0
    if not model_save_path.exists():
        model_save_path.mkdir(parents=True)
    for epoch in progress_bar(range(start_epoch, end_epoch)):
        print(f"epoch: {epoch}")
        model.train()
        for i, data in enumerate(train_data):
            optimizer.zero_grad()
            loss = learn_from_batch_ae(model, data, device)
            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()
            if lr_scheduler:
                lr_scheduler.step()
            loss_v = loss.item()
            losses.append(loss_v)
            it += 1
        loss_m = np.mean(losses)
        if loss_m < min_loss:
            min_loss_epoch = epoch
            min_loss = loss_m
        print(f'epoch: {epoch} loss: {loss_m}')
        losses = []
        torch.save(optimizer.state_dict(), str(model_save_path / "optimizer.pt"))
        torch.save(model.state_dict(), str(model_save_path / f"model-{epoch}.pt"))
    return dict(end_epoch=end_epoch, it=it, min_loss_epoch=min_loss_epoch)


In [7]:
import shutil


set_seed(123)
shutil.rmtree("./model", True)
shutil.rmtree("./logs", True)
save_path = Path("./model_prediction")
if not save_path.exists():
    save_path.mkdir(parents=True)

lr_scheduler = None
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AEModel()
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
res = dict(end_epoch=0, it=0, min_loss_epoch=0)
epochs = [5, 5, 5, 5]
for e in epochs:
    res = train_ae(model, loader0, optimizer, lr_scheduler, e, device=device,
                   start_epoch=res["end_epoch"], start_it=res["it"])
    res = train_ae(model, loader1, optimizer, lr_scheduler, e, device=device,
                   start_epoch=res["end_epoch"], start_it=res["it"])
    res = train_ae(model, loader2, optimizer, lr_scheduler, e, device=device,
                   start_epoch=res["end_epoch"], start_it=res["it"])

epoch = res["min_loss_epoch"]
shutil.copyfile(str(Path(MODEL_SAVE_PATH) / f"model-{epoch}.pt"), "ae-model.pt")

device: cuda


epoch: 0
epoch: 0 loss: 0.33238361266098526
epoch: 1
epoch: 1 loss: 0.1838451827827253
epoch: 2
epoch: 2 loss: 0.13273184334761218
epoch: 3
epoch: 3 loss: 0.09891085640380257
epoch: 4
epoch: 4 loss: 0.06769943737277859
device: cuda


epoch: 5
epoch: 5 loss: 0.04582122527062893
epoch: 6
epoch: 6 loss: 0.036750216782093045
epoch: 7
epoch: 7 loss: 0.03140839263796806
epoch: 8
epoch: 8 loss: 0.02712296452373266
epoch: 9
epoch: 9 loss: 0.02476088497787714
device: cuda


epoch: 10
epoch: 10 loss: 0.026889622805917515
epoch: 11
epoch: 11 loss: 0.02079536979819866
epoch: 12
epoch: 12 loss: 0.018061337596241463
epoch: 13
epoch: 13 loss: 0.016598807033547696
epoch: 14
epoch: 14 loss: 0.015628625222660127
device: cuda


epoch: 15
epoch: 15 loss: 0.020235618551898944
epoch: 16
epoch: 16 loss: 0.24000535277943863
epoch: 17
epoch: 17 loss: 0.08148313549004103
epoch: 18
epoch: 18 loss: 0.030957046406049477
epoch: 19
epoch: 19 loss: 0.0201392777656254
device: cuda


...

epoch: 40
epoch: 40 loss: 0.014685792273821984
epoch: 41
epoch: 41 loss: 0.013254401789225162
epoch: 42
epoch: 42 loss: 0.013171134714750534
epoch: 43
epoch: 43 loss: 0.012624278902373415
epoch: 44
epoch: 44 loss: 0.012258259143601073
device: cuda


epoch: 45
epoch: 45 loss: 0.007932240756130532
epoch: 46
epoch: 46 loss: 0.008110762414473453
epoch: 47
epoch: 47 loss: 0.007854008128082282
epoch: 48
epoch: 48 loss: 0.00747653864029991
epoch: 49
epoch: 49 loss: 0.007195870220465095
device: cuda


epoch: 50
epoch: 50 loss: 0.0058330844156444074
epoch: 51
epoch: 51 loss: 0.005697850743308664
epoch: 52
epoch: 52 loss: 0.005380401853471994
epoch: 53
epoch: 53 loss: 0.005222257366403938
epoch: 54
epoch: 54 loss: 0.005182330962270498
device: cuda


epoch: 55
epoch: 55 loss: 0.012163787010185261
epoch: 56
epoch: 56 loss: 0.011333895272555504
epoch: 57
epoch: 57 loss: 0.011080945861783433
epoch: 58
epoch: 58 loss: 0.01075590273087963
epoch: 59
epoch: 59 loss: 0.0104178328740787


'ae-model.pt'

### Training

In [8]:
def MCRMSE(y_true, y_pred):
    colwise_mse = torch.mean(torch.square(y_true - y_pred), dim=1)
    return torch.mean(torch.sqrt(colwise_mse), dim=1)


def sn_mcrmse_loss(predict, target, signal_to_noise):
    loss = MCRMSE(target, predict)
    weight = 0.5 * torch.log(signal_to_noise + 1.01)
    loss = (loss * weight).mean()
    return loss


def learn_from_batch(model, data, optimizer, lr_scheduler, device):
    optimizer.zero_grad()
    out = model(data["sequence"].to(device), data["bpp"].to(device))
    signal_to_noise = data["signal_to_noise"] * data["score"]
    loss = sn_mcrmse_loss(out, data["label"].to(device), signal_to_noise.to(device))
    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()
    if lr_scheduler:
        lr_scheduler.step()
    return out, loss


def evaluate(model, valid_data, device):
    model.eval()
    loss_list = []
    mcrmse = []
    for i, data in enumerate(valid_data):
        with torch.no_grad():
            y = model(data["sequence"].to(device), data["bpp"].to(device))
            mcrmse_ = MCRMSE(data["label"].to(device), y)[data["signal_to_noise"] > 1]
            mcrmse.append(mcrmse_.mean().item())
            loss = sn_mcrmse_loss(y, data["label"].to(device), data["signal_to_noise"].to(device))
            loss_list.append(loss.item())
    model.train()
    return dict(loss=np.mean(loss_list), mcmse=np.mean(mcrmse))


def train(model, train_data, valid_data, optimizer, lr_scheduler, epochs=10, device="cpu",
          start_epoch: int = 0, log_path: str = "./logs"):
    print(f"device: {device}")
    losses = []
    writer = SummaryWriter(log_path)
    it = 0
    model_save_path = Path(MODEL_SAVE_PATH)
    start_epoch = start_epoch
    end_epoch = start_epoch + epochs
    if not model_save_path.exists():
        model_save_path.mkdir(parents=True)
    min_eval_loss = 10.0
    min_eval_epoch = None
    for epoch in progress_bar(range(start_epoch, end_epoch)):
        print(f"epoch: {epoch}")
        model.train()
        for i, data in enumerate(train_data):
            _, loss = learn_from_batch(model, data, optimizer, lr_scheduler, device)
            loss_v = loss.item()
            writer.add_scalar('loss', loss_v, it)
            losses.append(loss_v)
            it += 1
        print(f'epoch: {epoch} loss: {np.mean(losses)}')
        losses = []

        eval_result = evaluate(model, valid_data, device)
        eval_loss = eval_result["loss"]
        if eval_loss <= min_eval_loss:
            min_eval_epoch = epoch
            min_eval_loss = eval_loss

        print(f"eval loss: {eval_loss} {eval_result['mcmse']}")
        writer.add_scalar(f"evaluate/loss", eval_loss, epoch)
        writer.add_scalar(f"evaluate/mcmse", eval_result["mcmse"], epoch)
        model.train()
        torch.save(optimizer.state_dict(), str(model_save_path / "optimizer.pt"))
        torch.save(model.state_dict(), str(model_save_path / f"model-{epoch}.pt"))
    print(f'min eval loss: {min_eval_loss} epoch {min_eval_epoch}')
    return min_eval_epoch


In [9]:
device = torch.device('cuda')
BATCH_SIZE = 64
base_train_data = pd.read_json(str(Path(BASE_PATH) / 'train.json'), lines=True)
samples = base_train_data
save_path = Path("./model_prediction")
if not save_path.exists():
    save_path.mkdir(parents=True)
shutil.rmtree("./model", True)
shutil.rmtree("./logs", True)
split = ShuffleSplit(n_splits=5, test_size=.1)
ids = samples.reset_index()["id"]
set_seed(124)
for fold, (train_index, test_index) in enumerate(split.split(samples)):
    print(f"fold: {fold}")
    train_df = samples.loc[train_index].reset_index()
    val_df = samples.loc[test_index].reset_index()
    train_loader = create_loader(train_df, BATCH_SIZE)
    valid_loader = create_loader(val_df, BATCH_SIZE)
    print(train_df.shape, val_df.shape)
    ae_model = AEModel()
    state_dict = torch.load("./ae-model.pt")
    ae_model.load_state_dict(state_dict)
    del state_dict
    model = FromAeModel(ae_model.seq)
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    lr_scheduler = None
    epoch = train(model, train_loader, valid_loader, optimizer, lr_scheduler, 200, device=device,
                  log_path=f"logs/{fold}")
    shutil.copyfile(str(Path(MODEL_SAVE_PATH) / f"./model-{epoch}.pt"), f"model_prediction/model-{fold}.pt")
    del model

fold: 0
(1, 107, 107, 3)
(1, 107, 107, 3)
(2160, 21) (240, 21)
device: cuda


epoch: 0
epoch: 0 loss: 0.3366467693731541
eval loss: 0.27427730942600126 0.35887618209466093
epoch: 1
epoch: 1 loss: 0.26438894328533524
eval loss: 0.2552827376947878 0.3338805421570049
epoch: 2
epoch: 2 loss: 0.2518751496919461
eval loss: 0.23802186024334912 0.31064318733379565
epoch: 3
epoch: 3 loss: 0.24164611418032653
eval loss: 0.2339008254297687 0.307056161498354
epoch: 4
epoch: 4 loss: 0.23753600060098226
eval loss: 0.23143972502903828 0.3029152419266091
epoch: 5
epoch: 5 loss: 0.23094102672851655
eval loss: 0.21992845045751575 0.29013867636456625
epoch: 6
epoch: 6 loss: 0.22358504679978777
eval loss: 0.21828351688292602 0.2861211594933033
epoch: 7
epoch: 7 loss: 0.217487902696557
eval loss: 0.2089559567556114 0.2751107299962422
epoch: 8
epoch: 8 loss: 0.21461846923621553
eval loss: 0.20489480210932562 0.2699206331893682
epoch: 9
epoch: 9 loss: 0.21076268213773208
eval loss: 0.20264072429394725 0.2668410091439313
epoch: 10
epoch: 10 loss: 0.20669808121460417
eval loss: 0.199998

epoch: 0
epoch: 0 loss: 0.3237348874634892
eval loss: 0.267901210924827 0.3435070239807859
epoch: 1
epoch: 1 loss: 0.250402792563676
eval loss: 0.2517359494497614 0.3241607510476169
epoch: 2
epoch: 2 loss: 0.23419194205055394
eval loss: 0.23315251662172315 0.2989475397733296
epoch: 3
epoch: 3 loss: 0.224133196725239
eval loss: 0.22621823318599665 0.2904107812266733
epoch: 4
epoch: 4 loss: 0.21832512190343967
eval loss: 0.23102457817585909 0.2955445131570566
epoch: 5
epoch: 5 loss: 0.21101670520856938
eval loss: 0.21536095901065574 0.2768328211433786
epoch: 6
epoch: 6 loss: 0.20500963820197876
eval loss: 0.21083532390805632 0.2717936258583198
epoch: 7
epoch: 7 loss: 0.19912293080868104
eval loss: 0.20318010775011086 0.26147565681886764
epoch: 8
epoch: 8 loss: 0.19140619332749753
eval loss: 0.20057776621924495 0.2579780276409592
epoch: 9
epoch: 9 loss: 0.18804296925326472
eval loss: 0.19970511261570756 0.2573978089354266
epoch: 10
epoch: 10 loss: 0.1844799461084573
eval loss: 0.191356178

epoch: 0
epoch: 0 loss: 0.30288508397713926
eval loss: 0.27280824375677293 0.3572874777679368
epoch: 1
epoch: 1 loss: 0.25314102947599154
eval loss: 0.2355632669102605 0.30969657309000287
epoch: 2
epoch: 2 loss: 0.2343748947123222
eval loss: 0.22509565940539675 0.29511758258655935
epoch: 3
epoch: 3 loss: 0.22394946591973539
eval loss: 0.21591301993318993 0.28240174024271103
epoch: 4
epoch: 4 loss: 0.2164217781294345
eval loss: 0.2085055256539137 0.2732869637255177
epoch: 5
epoch: 5 loss: 0.20961752224708124
eval loss: 0.20426365870541244 0.2678878940859002
epoch: 6
epoch: 6 loss: 0.20395572839623494
eval loss: 0.19935370479924427 0.2605347031378362
epoch: 7
epoch: 7 loss: 0.1972482326485436
eval loss: 0.19526278981227782 0.255661774341961
epoch: 8
epoch: 8 loss: 0.1939912875043874
eval loss: 0.1947706697224021 0.2559611571957996
epoch: 9
epoch: 9 loss: 0.1904241731957308
eval loss: 0.18881296624095034 0.2477819127476818
epoch: 10
epoch: 10 loss: 0.18459683918056796
eval loss: 0.1837790

epoch: 0
epoch: 0 loss: 0.3072507737134651
eval loss: 0.2529735538066676 0.33676025448655506
epoch: 1
epoch: 1 loss: 0.24443804015294504
eval loss: 0.23557113849225067 0.3129979616674715
epoch: 2
epoch: 2 loss: 0.23270223312400387
eval loss: 0.22480233897389762 0.3009121067290261
epoch: 3
epoch: 3 loss: 0.22230727566551986
eval loss: 0.21146490399776552 0.28178112954126433
epoch: 4
epoch: 4 loss: 0.21448810889466718
eval loss: 0.20702054663452768 0.27577840363490724
epoch: 5
epoch: 5 loss: 0.20589476284448224
eval loss: 0.19481388712882985 0.260287041597646
epoch: 6
epoch: 6 loss: 0.19935688129386714
eval loss: 0.19179725636495423 0.2535847156409439
epoch: 7
epoch: 7 loss: 0.1957186137363163
eval loss: 0.1911879773921579 0.2531527020389981
epoch: 8
epoch: 8 loss: 0.1927547360434159
eval loss: 0.18603754075367251 0.24723762076712985
epoch: 9
epoch: 9 loss: 0.18700385678367648
eval loss: 0.17889168599601032 0.23824694796906315
epoch: 10
epoch: 10 loss: 0.1824673532578965
eval loss: 0.184

epoch: 0
epoch: 0 loss: 0.31391567184749924
eval loss: 0.2553754534605806 0.3347165959338023
epoch: 1
epoch: 1 loss: 0.2482347459080892
eval loss: 0.23401207910973523 0.3078445428001895
epoch: 2
epoch: 2 loss: 0.23245708989545188
eval loss: 0.2242890014852031 0.29444974444147587
epoch: 3
epoch: 3 loss: 0.22323635739980272
eval loss: 0.21795180999063324 0.2863184049635539
epoch: 4
epoch: 4 loss: 0.21479934447259727
eval loss: 0.20841862279802592 0.27528799540954607
epoch: 5
epoch: 5 loss: 0.20964743952794745
eval loss: 0.2044964449142917 0.26953714219390074
epoch: 6
epoch: 6 loss: 0.2036397455523578
eval loss: 0.19787518912711383 0.260227498594348
epoch: 7
epoch: 7 loss: 0.19745111735220439
eval loss: 0.19423304800525476 0.2567984312168481
epoch: 8
epoch: 8 loss: 0.19381704016503662
eval loss: 0.19213820122937125 0.2546846697128869
epoch: 9
epoch: 9 loss: 0.18922561390455106
eval loss: 0.18667442117722507 0.246420795268034
epoch: 10
epoch: 10 loss: 0.18522044794119255
eval loss: 0.18782

In [10]:
def predict_batch(model, data, device):
    # batch x seq_len x target_size
    with torch.no_grad():
        pred = model(data["sequence"].to(device), data["bpp"].to(device))
        pred = pred.detach().cpu().numpy()
    return_values = []
    ids = data["ids"]
    for idx, p in enumerate(pred):
        id_ = ids[idx]
        assert p.shape == (model.pred_len, len(target_cols))
        for seqpos, val in enumerate(p):
            assert len(val) == len(target_cols)
            dic = {key: val for key, val in zip(target_cols, val)}
            dic["id_seqpos"] = f"{id_}_{seqpos}"
            return_values.append(dic)
    return return_values


def predict_data(model, loader, device, batch_size):
    data_list = []
    for i, data in enumerate(progress_bar(loader)):
        data_list += predict_batch(model, data, device)
    expected_length = model.pred_len * len(loader) * batch_size
    assert len(data_list) == expected_length, f"len = {len(data_list)} expected = {expected_length}"
    return data_list


### Prediction

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 1
base_test_data = pd.read_json(str(Path(BASE_PATH) / 'test.json'), lines=True)
public_df = base_test_data.query("seq_length == 107").copy()
private_df = base_test_data.query("seq_length == 130").copy()
print(f"public_df: {public_df.shape}")
print(f"private_df: {private_df.shape}")
public_df = public_df.reset_index()
private_df = private_df.reset_index()
pub_loader = create_loader(public_df, BATCH_SIZE, is_test=True)
pri_loader = create_loader(private_df, BATCH_SIZE, is_test=True)
pred_df_list = []
c = 0
for fold in range(5):
    model_load_path = f"./model_prediction/model-{fold}.pt"
    ae_model0 = AEModel()
    ae_model1 = AEModel()
    model_pub = FromAeModel(pred_len=107, seq=ae_model0.seq)
    model_pub = model_pub.to(device)
    model_pri = FromAeModel(pred_len=130, seq=ae_model1.seq)
    model_pri = model_pri.to(device)
    state_dict = torch.load(model_load_path, map_location=device)
    model_pub.load_state_dict(state_dict)
    model_pri.load_state_dict(state_dict)
    del state_dict

    data_list = []
    data_list += predict_data(model_pub, pub_loader, device, BATCH_SIZE)
    data_list += predict_data(model_pri, pri_loader, device, BATCH_SIZE)
    pred_df = pd.DataFrame(data_list, columns=["id_seqpos"] + target_cols)
    print(pred_df.head())
    print(pred_df.tail())
    pred_df_list.append(pred_df)
    c += 1
data_dic = dict(id_seqpos=pred_df_list[0]["id_seqpos"])
for col in target_cols:
    vals = np.zeros(pred_df_list[0][col].shape[0])
    for df in pred_df_list:
        vals += df[col].values
    data_dic[col] = vals / float(c)
pred_df_avg = pd.DataFrame(data_dic, columns=["id_seqpos"] + target_cols)
print(pred_df_avg.head())
pred_df_avg.to_csv("./submission.csv", index=False)


public_df: (629, 7)
private_df: (3005, 7)
(1, 107, 107, 3)
(1, 130, 130, 3)


        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.629786     0.630994  2.065533    0.580969  0.713464
1  id_00073f8be_1    1.851819     2.474012  3.514479    2.676078  2.341671
2  id_00073f8be_2    1.541160     0.570198  0.720891    0.691378  0.651467
3  id_00073f8be_3    1.288015     1.103547  1.088344    1.592761  1.770352
4  id_00073f8be_4    0.788181     0.522061  0.406513    0.739982  0.753662
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.370185     0.275616  0.265026    0.509553   
457949  id_ffda94f24_126    0.372463     0.510647  0.769562    0.789090   
457950  id_ffda94f24_127    0.714196     0.118798  0.132600    0.285008   
457951  id_ffda94f24_128    0.378564     0.056663  0.021460    0.304349   
457952  id_ffda94f24_129    0.358263     0.292962  0.241098    0.674212   

         deg_50C  
457948  0.173542  
457949  0.543501  
457950  0.274037  
457951 -0.011896  
4579

        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.670343     0.612814  2.063376    0.576358  0.809162
1  id_00073f8be_1    2.039258     2.979491  3.914047    3.176888  2.779592
2  id_00073f8be_2    1.639441     0.688121  0.902455    0.837290  0.802863
3  id_00073f8be_3    1.347732     1.139528  1.263196    1.631246  1.860349
4  id_00073f8be_4    0.868937     0.577777  0.466096    0.817850  0.808046
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.103262     0.348674  0.373398    0.476124   
457949  id_ffda94f24_126    0.296196     0.536590  0.684249    0.628332   
457950  id_ffda94f24_127    0.580862     0.140015  0.161873    0.270304   
457951  id_ffda94f24_128    0.066028     0.217901 -0.071348    0.202769   
457952  id_ffda94f24_129    0.145074     0.441275  0.406019    0.495075   

         deg_50C  
457948  0.428868  
457949  0.731486  
457950  0.527502  
457951  0.086912  
4579

        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.735993     0.617290  1.936633    0.488191  0.708700
1  id_00073f8be_1    2.739038     3.225997  4.149864    3.266102  2.955065
2  id_00073f8be_2    1.710197     0.579142  0.518898    0.593052  0.773471
3  id_00073f8be_3    1.417613     1.318627  1.367229    1.889270  2.151289
4  id_00073f8be_4    0.992409     0.497510  0.485291    0.675799  0.895142
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.117115     0.382759  0.330601    0.612530   
457949  id_ffda94f24_126    0.221809     0.448559  0.805866    0.707057   
457950  id_ffda94f24_127    0.451916     0.267606  0.256535    0.269197   
457951  id_ffda94f24_128    0.194830     0.255570  0.180311    0.300914   
457952  id_ffda94f24_129    0.156188     0.273056  0.380679    0.386971   

         deg_50C  
457948  0.311062  
457949  0.743625  
457950  0.473123  
457951  0.156445  
4579

        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.713856     0.720683  1.945832    0.514947  0.722098
1  id_00073f8be_1    2.025032     2.916558  4.338650    3.043663  2.812816
2  id_00073f8be_2    1.526654     0.601659  0.636573    0.593480  0.573333
3  id_00073f8be_3    1.326741     0.997878  1.191320    1.417414  1.765372
4  id_00073f8be_4    0.800972     0.643771  0.549665    0.890862  0.802686
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.118192     0.615466  0.506147    0.724437   
457949  id_ffda94f24_126    0.356832     0.466941  0.764185    0.639365   
457950  id_ffda94f24_127    0.446695     0.207680  0.284433    0.254582   
457951  id_ffda94f24_128    0.185936     0.039250  0.082429    0.225192   
457952  id_ffda94f24_129    0.220845     0.432117  0.504581    0.472217   

         deg_50C  
457948  0.292530  
457949  0.542255  
457950  0.322626  
457951  0.115039  
4579

        id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C   deg_50C
0  id_00073f8be_0    0.728909     0.684135  1.956060    0.526627  0.772040
1  id_00073f8be_1    2.518496     3.621165  4.089748    3.430235  2.815590
2  id_00073f8be_2    1.630907     0.633605  0.739487    0.725516  0.709910
3  id_00073f8be_3    1.366814     1.077618  1.294116    1.533260  1.893265
4  id_00073f8be_4    0.863779     0.586744  0.507793    0.877569  0.897555
               id_seqpos  reactivity  deg_Mg_pH10  deg_pH10  deg_Mg_50C  \
457948  id_ffda94f24_125    0.057121     0.326073  0.325887    0.538597   
457949  id_ffda94f24_126    0.172986     0.295986  0.493952    0.363920   
457950  id_ffda94f24_127    0.498051     0.146676  0.238410    0.229348   
457951  id_ffda94f24_128    0.114384     0.156119  0.109840    0.329870   
457952  id_ffda94f24_129    0.098567     0.399737  0.181518    0.319849   

         deg_50C  
457948  0.195759  
457949  0.359727  
457950  0.259736  
457951 -0.013004  
4579